# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [ ]:
#Es necesario un procesamiento previo de las cadenas (docs) para eliminar los signos de puntación y cáracteres especiales, como, saltos de linea.
import re

def limpiar_doc (doc):
    regex = r"[,.;'\"\n]"

    return re.sub(regex, "", doc)

# Prueba
corpus = open("01_corpus_turismo.txt", encoding="utf-8")

doc = corpus.readline()
doc_limpio = limpiar_doc(doc)

corpus.close()


print (doc)
print (doc_limpio)

Ecuador es un país megadiverso, con playas, montañas y selvas ideales para el turismo ecológico.

Ecuador es un país megadiverso con playas montañas y selvas ideales para el turismo ecológico


In [166]:
def buscar_palabra_lineal (query, corpus_path, encoding='utf-8', case_insensitive=False):
    resultados = set([])

    if case_insensitive:
        query = query.lower()

    corpus = open(corpus_path, 'r', encoding=encoding)

    for i, doc in enumerate(corpus, start=1):
        doc = limpiar_doc(doc).split(" ")

        if case_insensitive:
            doc = map(str.lower, doc)

        if query in doc:
            resultados.add(i)
    
    corpus.close()
    
    return resultados

In [167]:
def imprimir_resultado (query, corpus_path, resultado):
    print (f"Corpus: {corpus_path}")
    print (f"La palabra '{query}' se encuentra {len(resultado)} veces en el corpus, en documentos: {resultado}")

query = "Quito"
#T1:
path = "01_corpus_turismo.txt"
imprimir_resultado(query, path, buscar_palabra_lineal(query, path,case_insensitive=True))
imprimir_resultado(query, path, buscar_palabra_lineal(query, path,case_insensitive=False))

#T2:
path2 = "01_corpus_turismo_500.txt"
imprimir_resultado(query, path2, buscar_palabra_lineal(query, path2))

Corpus: 01_corpus_turismo.txt
La palabra 'Quito' se encuentra 2 veces en el corpus, en documentos: {19, 3}
Corpus: 01_corpus_turismo.txt
La palabra 'Quito' se encuentra 1 veces en el corpus, en documentos: {3}
Corpus: 01_corpus_turismo_500.txt
La palabra 'Quito' se encuentra 20 veces en el corpus, en documentos: {4, 260, 391, 136, 16, 17, 291, 39, 43, 325, 73, 201, 75, 213, 469, 89, 355, 487, 104, 238}


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [168]:
def generar_indice_invertido(corpus_path, encoding="utf-8"):
    #Construir un indice invertido
    indice_invertido = {}

    corpus = open(corpus_path, 'r', encoding=encoding)

    for i, doc in  enumerate(corpus, start=1):
        
        doc = limpiar_doc(doc).split(" ")

        for palabra in doc:
            if palabra in indice_invertido:
                indice_invertido[palabra].add(i)
            else:
                indice_invertido[palabra] = set([i])

    corpus.close()
    return indice_invertido


#Ejemplo con query
query = "UNESCO"
path = "01_corpus_turismo.txt"
indice_invertido = generar_indice_invertido(path)

print (indice_invertido)
print (f"La palabra {query} se encuentra en: {indice_invertido.get(query, set([]))}")


{'Ecuador': {1}, 'es': {1, 6, 9}, 'un': {1, 3, 9}, 'país': {1}, 'megadiverso': {1}, 'con': {1, 3, 7, 10, 11}, 'playas': {1, 10, 5}, 'montañas': {1}, 'y': {1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16}, 'selvas': {1}, 'ideales': {1}, 'para': {1, 12, 9}, 'el': {1, 2, 9, 12, 14, 15}, 'turismo': {1}, 'ecológico': {1}, 'Las': {2}, 'Islas': {2}, 'Galápagos': {2}, 'atraen': {2}, 'visitantes': {2, 15}, 'de': {2, 3, 4, 8, 9, 11, 14, 15, 16}, 'todo': {2}, 'mundo': {2}, 'por': {2, 3, 6}, 'su': {2, 6, 7, 10, 11}, 'biodiversidad': {2}, 'única': {2}, 'La': {11, 3, 5}, 'ciudad': {3}, 'Quito': {3}, 'cuenta': {3}, 'centro': {3}, 'histórico': {3}, 'declarado': {3}, 'Patrimonio': {3}, 'la': {3, 9, 12, 14, 15}, 'Humanidad': {3}, 'UNESCO': {3}, 'En': {4, 12}, 'Baños': {4}, 'Agua': {4}, 'Santa': {4}, 'los': {4, 7, 8, 13, 16}, 'turistas': {4, 13, 14, 7}, 'disfrutan': {4, 14}, 'deportes': {4}, 'aventura': {4}, 'como': {4, 5, 15}, 'rafting': {4}, 'canopy': {4}, 'Ruta': {5}, 'del': {10, 11, 5}, 'Spondylus': {5}

In [169]:
#Función de búsqueda
def buscar_palabra_ii (query, indice_invertido=indice_invertido): 
    return indice_invertido.get(query, set([])) # se pasa un set vacío para aquellas palabras que no existen en el indice, estas no son parte del texto 



#Ejemplo
query = "López"
imprimir_resultado(query, path, buscar_palabra_ii(query,indice_invertido=indice_invertido))



Corpus: 01_corpus_turismo.txt
La palabra 'López' se encuentra 1 veces en el corpus, en documentos: {5}


## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [170]:
#Se genera el indice invertido para el corpus 01_corpus_turismo_500.txt
indice_invertido_500 = generar_indice_invertido("01_corpus_turismo_500.txt")

print(indice_invertido_500)

{'Otavalo': {1, 7, 149, 29, 159, 161, 34, 297, 57, 60, 191, 65, 193, 204, 334, 207, 335, 340, 342, 216, 222, 351, 101, 358, 488, 363, 111, 367, 496, 373, 118, 119, 380}, 'es': {1, 5, 7, 8, 10, 14, 22, 29, 30, 32, 33, 34, 37, 41, 45, 53, 57, 60, 65, 72, 76, 78, 101, 106, 109, 110, 111, 115, 118, 119, 122, 127, 130, 138, 149, 154, 156, 159, 161, 163, 168, 171, 177, 180, 187, 190, 191, 193, 200, 204, 207, 212, 216, 222, 223, 225, 231, 246, 248, 250, 254, 256, 258, 264, 271, 275, 280, 289, 293, 294, 297, 307, 309, 319, 324, 331, 333, 334, 335, 340, 341, 342, 345, 349, 351, 352, 358, 363, 365, 367, 371, 373, 379, 380, 383, 386, 388, 410, 412, 420, 437, 442, 447, 448, 454, 456, 459, 460, 462, 463, 465, 468, 477, 482, 486, 488, 491, 492, 496}, 'conocido': {1, 7, 149, 29, 159, 161, 34, 297, 57, 60, 191, 65, 193, 204, 334, 207, 335, 340, 342, 216, 222, 351, 101, 358, 488, 363, 111, 367, 496, 373, 118, 119, 380}, 'por': {1, 2, 7, 10, 12, 14, 23, 24, 25, 29, 31, 32, 34, 35, 42, 47, 48, 57, 60, 61

In [171]:
# 1. Búsqueda de varias palabras
palabras_a_buscar = ['quito', 'montaña', 'feriado', 'playas', 'aventura', 'galápagos']

def busqueda_de_palabras (palabras, corpus_o_ii, busqueda):
    resultados = {}

    for palabra in palabras:
        resultados[palabra] = busqueda(palabra, corpus_o_ii)

    return resultados



In [172]:
# Implementación con corpus pequeño
corpus= "01_corpus_turismo.txt"
busqueda_lineal = busqueda_de_palabras(palabras_a_buscar, corpus, buscar_palabra_lineal)
busqueda_ii = busqueda_de_palabras(palabras_a_buscar, indice_invertido, buscar_palabra_ii)

print(f"Busqueda en corpus {corpus}")
print (f"Lineal: {busqueda_lineal}")
print (f"Indice: {busqueda_ii}")

Busqueda en corpus 01_corpus_turismo.txt
Lineal: {'quito': {19}, 'montaña': set(), 'feriado': {12, 14}, 'playas': {1, 10, 5}, 'aventura': {4}, 'galápagos': set()}
Indice: {'quito': {19}, 'montaña': set(), 'feriado': {12, 14}, 'playas': {1, 10, 5}, 'aventura': {4}, 'galápagos': set()}


In [173]:
# Implementación con corpus grande
corpus = "01_corpus_turismo_500.txt"
busqueda_lineal = busqueda_de_palabras(palabras_a_buscar, corpus, buscar_palabra_lineal)
busqueda_ii = busqueda_de_palabras(palabras_a_buscar, indice_invertido_500, buscar_palabra_ii)

print(f"Busqueda en corpus {corpus}")
print (f"Lineal: {busqueda_lineal}")
print (f"Indice: {busqueda_ii}")

Busqueda en corpus 01_corpus_turismo_500.txt
Lineal: {'quito': set(), 'montaña': set(), 'feriado': {9, 10, 12, 16, 27, 30, 32, 33, 34, 35, 36, 40, 44, 50, 56, 65, 66, 70, 73, 75, 79, 80, 91, 106, 108, 109, 110, 113, 116, 123, 124, 130, 132, 133, 134, 138, 139, 147, 148, 153, 154, 155, 157, 162, 166, 167, 168, 170, 179, 181, 189, 192, 194, 195, 199, 200, 202, 203, 205, 210, 211, 213, 217, 223, 225, 229, 233, 234, 248, 257, 262, 267, 269, 271, 280, 281, 282, 286, 295, 303, 306, 308, 309, 310, 311, 315, 318, 321, 325, 328, 329, 331, 333, 337, 340, 342, 344, 346, 351, 354, 360, 361, 362, 365, 369, 372, 374, 375, 377, 380, 383, 390, 395, 406, 408, 410, 412, 421, 429, 437, 440, 441, 443, 447, 448, 451, 457, 459, 470, 473, 474, 475, 480, 482, 487, 492, 494, 495}, 'playas': {384, 390, 135, 266, 396, 273, 20, 277, 405, 151, 406, 28, 417, 419, 36, 164, 423, 170, 298, 299, 430, 51, 321, 322, 67, 69, 198, 327, 328, 79, 83, 467, 87, 219, 96, 480, 100, 229, 103, 233, 235, 236, 239, 370, 244, 120, 12

In [174]:
#2. Modificar el código para medir el tiempo
import time #segundos con precisión de microsegundos

def busqueda_de_palabras (palabras, corpus_o_ii, busqueda, medirTiempo=False):
    resultados = {}

    for palabra in palabras:
        if medirTiempo:
            inicio = time.time()
            resultado = busqueda(palabra, corpus_o_ii)
            fin = time.time()

            tiempo_ejecucion=fin-inicio
            
            resultados[palabra] = [resultado, tiempo_ejecucion]
        else:
            resultados[palabra] = busqueda(palabra, corpus_o_ii)

    return resultados

In [175]:
# 2.1. lineal
corpus= "01_corpus_turismo_500.txt"
busqueda_lineal = busqueda_de_palabras(palabras_a_buscar, corpus, buscar_palabra_lineal, medirTiempo=True)
busqueda_ii = busqueda_de_palabras(palabras_a_buscar, indice_invertido_500, buscar_palabra_ii, medirTiempo=True)

print(f"Busqueda en corpus {corpus}")
print (f"Lineal: {busqueda_lineal}")


Busqueda en corpus 01_corpus_turismo_500.txt
Lineal: {'quito': [set(), 0.0036821365356445312], 'montaña': [set(), 0.0038750171661376953], 'feriado': [{9, 10, 12, 16, 27, 30, 32, 33, 34, 35, 36, 40, 44, 50, 56, 65, 66, 70, 73, 75, 79, 80, 91, 106, 108, 109, 110, 113, 116, 123, 124, 130, 132, 133, 134, 138, 139, 147, 148, 153, 154, 155, 157, 162, 166, 167, 168, 170, 179, 181, 189, 192, 194, 195, 199, 200, 202, 203, 205, 210, 211, 213, 217, 223, 225, 229, 233, 234, 248, 257, 262, 267, 269, 271, 280, 281, 282, 286, 295, 303, 306, 308, 309, 310, 311, 315, 318, 321, 325, 328, 329, 331, 333, 337, 340, 342, 344, 346, 351, 354, 360, 361, 362, 365, 369, 372, 374, 375, 377, 380, 383, 390, 395, 406, 408, 410, 412, 421, 429, 437, 440, 441, 443, 447, 448, 451, 457, 459, 470, 473, 474, 475, 480, 482, 487, 492, 494, 495}, 0.0024471282958984375], 'playas': [{384, 390, 135, 266, 396, 273, 20, 277, 405, 151, 406, 28, 417, 419, 36, 164, 423, 170, 298, 299, 430, 51, 321, 322, 67, 69, 198, 327, 328, 79, 83,

In [176]:
busqueda_ii = busqueda_de_palabras(palabras_a_buscar, indice_invertido_500, buscar_palabra_ii, medirTiempo=True)

print(f"Busqueda en corpus {corpus}")
print (f"usando indice: {busqueda_ii}")

Busqueda en corpus 01_corpus_turismo_500.txt
usando indice: {'quito': [set(), 2.6226043701171875e-06], 'montaña': [set(), 9.5367431640625e-07], 'feriado': [{9, 10, 12, 16, 27, 30, 32, 33, 34, 35, 36, 40, 44, 50, 56, 65, 66, 70, 73, 75, 79, 80, 91, 106, 108, 109, 110, 113, 116, 123, 124, 130, 132, 133, 134, 138, 139, 147, 148, 153, 154, 155, 157, 162, 166, 167, 168, 170, 179, 181, 189, 192, 194, 195, 199, 200, 202, 203, 205, 210, 211, 213, 217, 223, 225, 229, 233, 234, 248, 257, 262, 267, 269, 271, 280, 281, 282, 286, 295, 303, 306, 308, 309, 310, 311, 315, 318, 321, 325, 328, 329, 331, 333, 337, 340, 342, 344, 346, 351, 354, 360, 361, 362, 365, 369, 372, 374, 375, 377, 380, 383, 390, 395, 406, 408, 410, 412, 421, 429, 437, 440, 441, 443, 447, 448, 451, 457, 459, 470, 473, 474, 475, 480, 482, 487, 492, 494, 495}, 9.5367431640625e-07], 'playas': [{384, 390, 135, 266, 396, 273, 20, 277, 405, 151, 406, 28, 417, 419, 36, 164, 423, 170, 298, 299, 430, 51, 321, 322, 67, 69, 198, 327, 328, 79,

In [177]:
!pip install pandas

In [178]:
#2.3.Presentar una tabla de resultados
corpus= "01_corpus_turismo_500.txt"

# generar valores
busqueda_lineal = busqueda_de_palabras(palabras_a_buscar, corpus, buscar_palabra_lineal, medirTiempo=True)
busqueda_ii = busqueda_de_palabras(palabras_a_buscar, indice_invertido_500, buscar_palabra_ii, medirTiempo=True)

In [179]:
# generar tabla con pandas
# palabras/busqueda | lineal | invertida |
# palala1           |        |           |

#print (busqueda_ii)
#print( busqueda_lineal)

resultados = []
aux = [0,0]
for palabra in palabras_a_buscar:
    valor_bl = busqueda_lineal[palabra][1]
    valor_ii = busqueda_ii[palabra][1]
    resultados.append([valor_bl*1000, valor_ii*1000 , busqueda_ii[palabra][0]])
    aux[0]+=valor_bl*1000
    aux[1]+=valor_ii*1000

resultados.append([aux[0], aux[1], ""])

#print (resultados)


#Uso de DateFrame para presentar la tabla con pandas
import pandas as pd

filas = palabras_a_buscar.copy()
filas.append("Total")
columnas = ["Búsqueda Lineal [ms]", "Busqueda usando índice invertido [ms]", "Documentos"]

df = pd.DataFrame(resultados, columns=columnas, index=filas)

print (df)
    

           Búsqueda Lineal [ms]  Busqueda usando índice invertido [ms]  \
quito                  4.005671                               0.002146   
montaña                2.436399                               0.000715   
feriado                2.317905                               0.000954   
playas                 2.188921                               0.000715   
aventura               2.204895                               0.000715   
galápagos              2.429485                               0.000477   
Total                 15.583277                               0.005722   

                                                  Documentos  
quito                                                     {}  
montaña                                                   {}  
feriado    {9, 10, 12, 16, 27, 30, 32, 33, 34, 35, 36, 40...  
playas     {384, 390, 135, 266, 396, 273, 20, 277, 405, 1...  
aventura   {389, 392, 276, 278, 408, 284, 38, 54, 311, 43...  
galápagos                    

## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [180]:
#1. Modificar la creación del índice invertido para que sea insensible a las mayús
def generar_indice_invertido(corpus_path, encoding="utf-8", case_insesitive = False):
    #Construir un indice invertido
    indice_invertido = {}

    corpus = open(corpus_path, 'r', encoding=encoding)

    for i, doc in  enumerate(corpus, start=1):
        
        doc = limpiar_doc(doc).split(" ")

        for palabra in doc:
            if case_insesitive:
                palabra = palabra.lower()   # se utiliza lower de str para que todas las cadenas solo usen minusculas, esto tambien cambia la busqueda

            if palabra in indice_invertido:
                indice_invertido[palabra].add(i)
            else:
                indice_invertido[palabra] = set([i])

    corpus.close()
    return indice_invertido

def buscar_palabra_ii (query, indice_invertido, case_insensitive = False):

    if case_insensitive:
        query = query.lower()
     
    return indice_invertido.get(query, set([])) 

In [181]:
#Generar los indices
indice_invertido = generar_indice_invertido("01_corpus_turismo.txt",case_insesitive=True)
indice_invertido_500 = generar_indice_invertido ("01_corpus_turismo_500.txt", case_insesitive=True)

print (indice_invertido)
print(indice_invertido_500)

{'ecuador': {1}, 'es': {1, 6, 9}, 'un': {1, 3, 9}, 'país': {1}, 'megadiverso': {1}, 'con': {1, 3, 7, 10, 11}, 'playas': {1, 10, 5}, 'montañas': {1}, 'y': {1, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16}, 'selvas': {1}, 'ideales': {1}, 'para': {1, 12, 9}, 'el': {1, 2, 8, 9, 12, 13, 14, 15}, 'turismo': {1}, 'ecológico': {1}, 'las': {8, 2, 10, 14}, 'islas': {2}, 'galápagos': {2}, 'atraen': {2}, 'visitantes': {2, 15}, 'de': {2, 3, 4, 8, 9, 11, 14, 15, 16}, 'todo': {2}, 'mundo': {2}, 'por': {2, 3, 6}, 'su': {2, 6, 7, 10, 11}, 'biodiversidad': {2}, 'única': {2}, 'la': {3, 5, 9, 11, 12, 14, 15}, 'ciudad': {3}, 'quito': {19, 3}, 'cuenta': {3}, 'centro': {3}, 'histórico': {3}, 'declarado': {3}, 'patrimonio': {3}, 'humanidad': {3}, 'unesco': {3}, 'en': {4, 12, 14, 15}, 'baños': {4}, 'agua': {4}, 'santa': {4}, 'los': {4, 7, 8, 13, 14, 16}, 'turistas': {4, 13, 14, 7}, 'disfrutan': {4, 14}, 'deportes': {4}, 'aventura': {4}, 'como': {4, 5, 15}, 'rafting': {4}, 'canopy': {4}, 'ruta': {5}, 'del': {10, 1

In [182]:
# 2. Consulta para múltiples términos

#Para busqueda de indice invertido
def busqueda_multitermino (palabras,  corpus_o_ii, busqueda, case_insensitive=False):
    resultados = []

    #Obtener resultados
    for palabra in palabras: 
        resultados.append(busqueda(palabra, corpus_o_ii, case_insensitive=case_insensitive))

    # Realizar la intersección de conjuntos
    res = resultados[0]
    for i in range(len(resultados)-1):
        res = res.intersection(resultados[i+1])
    
    return res

In [183]:
palabras = ["megadiverso", "Ecuador", "ecológico"]
corpus = "01_corpus_turismo.txt"

busqueda_lineal = busqueda_multitermino(palabras, corpus, buscar_palabra_lineal, case_insensitive=True)
busqueda_ii = busqueda_multitermino(palabras, indice_invertido, buscar_palabra_ii, case_insensitive=True)

print(f"Búsqueda Lineal: {busqueda_lineal}")
print(f"Búsqueda usando índice invertido: {busqueda_ii}")

Búsqueda Lineal: {1}
Búsqueda usando índice invertido: {1}


In [184]:
palabras = ["megadiverso", "Ecuador", "ecológico"]

corpus = "01_corpus_turismo_500.txt"

busqueda_lineal = busqueda_multitermino(palabras, corpus, buscar_palabra_lineal, case_insensitive=True)
busqueda_ii = busqueda_multitermino(palabras, indice_invertido_500, buscar_palabra_ii, case_insensitive=True)

print(f"Búsqueda Lineal encuentra {len(busqueda_lineal)}: {busqueda_lineal}")
print(f"Búsqueda usando índice invertido encuentra {len(busqueda_ii)}: {busqueda_ii}")

Búsqueda Lineal encuentra 36: {130, 258, 386, 5, 22, 280, 156, 30, 163, 37, 168, 41, 45, 307, 442, 319, 324, 200, 459, 76, 333, 352, 225, 486, 231, 491, 492, 109, 110, 371, 246, 122, 379, 127, 254, 383}
Búsqueda usando índice invertido encuentra 36: {130, 258, 386, 5, 22, 280, 156, 30, 163, 37, 168, 41, 45, 307, 442, 319, 324, 200, 459, 76, 333, 352, 225, 486, 231, 491, 492, 109, 110, 371, 246, 122, 379, 127, 254, 383}


In [185]:
#3. Cálculo del SpeedUp
#Se pretende calucular el speedup entre la búsqueda lineal y con índice invertido; en la búsqueda simple y no multi_término

query = ["disfrutan"]
corpus = "01_corpus_turismo_500.txt"
ii=indice_invertido_500
cantidad = 10

busquedas_lineales = []
busquedas_ii = []

for i in range(cantidad):
    busquedas_lineales.append(busqueda_de_palabras(query,corpus,buscar_palabra_lineal,medirTiempo=True))
    busquedas_ii.append(busqueda_de_palabras(query,ii,buscar_palabra_ii, medirTiempo=True))

print (busquedas_lineales)
print()
print (busquedas_ii)


#se obtiene un valor promedio en ms
valor_bl = 0
valor_ii = 0

for i in range (cantidad):
    valor_bl+= busquedas_lineales[i][query[0]][1]
    valor_ii+=busquedas_ii[i][query[0]][1]
else:
    valor_bl = valor_bl/cantidad *1000
    valor_ii = valor_ii/cantidad * 1000

print("_____________________________________________________________________________________________________")
print (f"Tiempo promedio [ms] de búsqueda lineal: {valor_bl}")
print (f"Tiempo promedio [ms] de búsqueda con índice invertido: {valor_ii}")

#calculo de speedup
speedup =  valor_bl/valor_ii 
print("_____________________________________________________________________________________________________")
print (f"SpeedUp de Búsqueda Lineal/Busqueda con índice invertido: {speedup}")
print (f"Interpretación: La búsqueda con índice invertido es {round(speedup,2)} más rápida que la búsqueda lineal.")

[{'disfrutan': [{129, 6, 393, 395, 140, 397, 143, 144, 401, 282, 286, 418, 292, 174, 303, 431, 433, 50, 183, 58, 320, 455, 74, 215, 473, 226, 228, 102, 108, 364, 240, 241}, 0.0029680728912353516]}, {'disfrutan': [{129, 6, 393, 395, 140, 397, 143, 144, 401, 282, 286, 418, 292, 174, 303, 431, 433, 50, 183, 58, 320, 455, 74, 215, 473, 226, 228, 102, 108, 364, 240, 241}, 0.0024137496948242188]}, {'disfrutan': [{129, 6, 393, 395, 140, 397, 143, 144, 401, 282, 286, 418, 292, 174, 303, 431, 433, 50, 183, 58, 320, 455, 74, 215, 473, 226, 228, 102, 108, 364, 240, 241}, 0.0024280548095703125]}, {'disfrutan': [{129, 6, 393, 395, 140, 397, 143, 144, 401, 282, 286, 418, 292, 174, 303, 431, 433, 50, 183, 58, 320, 455, 74, 215, 473, 226, 228, 102, 108, 364, 240, 241}, 0.0026268959045410156]}, {'disfrutan': [{129, 6, 393, 395, 140, 397, 143, 144, 401, 282, 286, 418, 292, 174, 303, 431, 433, 50, 183, 58, 320, 455, 74, 215, 473, 226, 228, 102, 108, 364, 240, 241}, 0.002461671829223633]}, {'disfrutan': [